In [1]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 164.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import urllib

import boto3
import torch
from torchvision import transforms
from PIL import Image

In [3]:
model = torch.hub.load('nicolalandro/ntsnet-cub200', 'ntsnet', pretrained=True,
                       **{'topN': 6, 'device':'cpu', 'num_classes': 200})

Using cache found in /opt/app-root/src/.cache/torch/hub/nicolalandro_ntsnet-cub200_master


In [21]:
transform_test = transforms.Compose([
    transforms.Resize((600, 600), Image.BILINEAR),
    transforms.CenterCrop((448, 448)),
    # transforms.RandomHorizontalFlip(),  # only if train
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])


model = torch.hub.load('nicolalandro/ntsnet-cub200', 'ntsnet', pretrained=True, **{'topN': 6, 'device':'cpu', 'num_classes': 200})
model.eval()

url = 'https://upload.wikimedia.org/wikipedia/commons/d/d8/Cardinalis_cardinalis_-Columbus%2C_Ohio%2C_USA-male-8_%281%29.jpg'
img = Image.open(urllib.request.urlopen(url))
scaled_img = transform_test(img)
torch_images = scaled_img.unsqueeze(0)
with torch.no_grad():
    top_n_coordinates, concat_out, raw_logits, concat_logits, part_logits, top_n_index, top_n_prob = model(torch_images)

    _, predict = torch.max(concat_logits, 1)
    pred_id = predict.item()
    print("bird class id:", pred_id)
    print('bird class:', model.bird_classes[pred_id])

Using cache found in /opt/app-root/src/.cache/torch/hub/nicolalandro_ntsnet-cub200_master


bird class id: 16
bird class: 017.Cardinal


In [14]:
# Convert to ONNX

onnx_filename = "birds_classifier1.onnx"
input_names = ["torch_images"]
output_names = ["output"]
# ONNX to infers the input shape from an example
# A list of RGB 448x448 images
example_input = torch.randn(1, 3, 448, 448)

torch.onnx.export(model, example_input, onnx_filename, input_names=input_names, output_names=["top_n_coordinates", "concat_out", "raw_logits", "concat_logits", "part_logits", "top_n_index", "top_n_prob"])

In [15]:
key_id = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
region = os.getenv("AWS_DEFAULT_REGION")
endpoint = os.getenv("AWS_S3_ENDPOINT")
bucket_name = os.getenv("AWS_S3_BUCKET")

s3_client = boto3.client(
    "s3",
    region,
    aws_access_key_id=key_id,
    aws_secret_access_key=secret_key,
    endpoint_url=endpoint,
    use_ssl=True
)

s3_client.upload_file(onnx_filename, bucket_name, Key=onnx_filename)

print(f"File {onnx_filename} uploaded to S3!")

File birds_classifier1.onnx uploaded to S3!
